<a href="https://colab.research.google.com/github/KevinYih/BigDataDemo/blob/main/Copy_of_SVD_Netflix_movie_ratings_by_Kevin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SVD - Netflix movie ratings

Adapted from: https://colab.research.google.com/github/ashishdasari148/Recommender-Systems-using-Collaborative-Filtering/blob/master/Netflix.ipynb#scrollTo=qX7Nk4tUGZND

In [1]:
!echo '{"username":"kevinyih","key":"9a18a73669b892b2ac58ebb67dc96545"}' > /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!pip install kaggle
!kaggle datasets download -d netflix-inc/netflix-prize-data

/bin/bash: line 1: /root/.kaggle/kaggle.json: No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data
License(s): other
100% 683M/683M [00:26<00:00, 29.1MB/s]
100% 683M/683M [00:26<00:00, 26.8MB/s]


In [2]:
import os
import zipfile
with zipfile.ZipFile('netflix-prize-data.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
import pandas as pd

import re
import os
import time
import matplotlib.pyplot as plt
import numpy as np
dir_name = 'data'

In [4]:
# Converting the txt files into csv files
files = ['combined_data_1.txt','combined_data_2.txt','combined_data_3.txt','combined_data_4.txt']
for j in files:
    file1 = open(os.path.join(dir_name, j),"r+")
    print(j + " opened...")
    file2 = open(os.path.join(dir_name, j + '.csv'),"a")
    for i in file1.readlines():
        temp = re.match("(\d*)[:]", i)
        if(temp):
            movienumber = temp.group(1)
        else:
            file2.write(str(movienumber)+","+i)
    file1.close()
    file2.close()
    print(j + " closed...")

combined_data_1.txt opened...
combined_data_1.txt closed...
combined_data_2.txt opened...
combined_data_2.txt closed...
combined_data_3.txt opened...
combined_data_3.txt closed...
combined_data_4.txt opened...
combined_data_4.txt closed...


In [5]:
movie_titles = pd.read_csv(os.path.join(dir_name, 'movie_titles.csv'), encoding = 'ISO-8859-1', header = None, names=['Movie', 'Year of Release', 'Title'], usecols=[0, 1, 2])
movie_titles

,Movie,Year of Release,Title
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
...,...,...,...
17765,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004.0,Fidel Castro: American Experience
17767,17768,2000.0,Epoch
17768,17769,2003.0,The Company


In [6]:
## appending all the csv files of rating data into a single dataframe
dfs = []
for i in ['combined_data_1.txt.csv','combined_data_2.txt.csv','combined_data_3.txt.csv','combined_data_4.txt.csv']:
    df_ = pd.read_csv(os.path.join(dir_name, i), header=None, names=['Movie', 'User', 'Rating', 'Date'], usecols=[0, 1, 2, 3])
    df_ = df_[['Movie','User','Rating']]
    dfs.append(df_)
df_all = pd.concat(dfs, ignore_index=True)

In [7]:
print("number of ratings:", len(df_all))

number of ratings: 100480507


In [8]:
# search for your favorite movie

indices = movie_titles.Title.str.contains('dinosaur', flags=re.IGNORECASE)
movie_titles[indices]
# 14691    The Matrix
# 3925     The Matrix: Reloaded
# 16767    The Matrix: Revisited
# 468      The Matrix: Revolutions
# 397      A Night in Casablanca
# 10661    Tokyo Story
# 963      A Streetcar Named Desire
# 6099     Apocalypse Now
# 4253    Kind Hearts and Coronets
# 2412     Harry Potter and the Prisoner of Azkaban: Bonu...
# 11443    Harry Potter and the Chamber of Secrets



,Movie,Year of Release,Title
0,1,2003.0,Dinosaur Planet
650,651,1997.0,The Rainbow Fish / Dazzle the Dinosaur
693,694,2000.0,When Dinosaurs Roamed America
1172,1173,1999.0,Walking with Dinosaurs
1882,1883,1960.0,Dinosaurus!
4099,4100,2000.0,Dinosaur
5301,5302,2003.0,Chased by Dinosaurs: Three Walking with Dinosa...
8841,8842,1999.0,Walking with Dinosaurs: Bonus Material
13585,13586,2001.0,Allosaurus: A Walking with Dinosaurs Special
13890,13891,1991.0,Nymphoid Barbarian in Dinosaur Hell


In [9]:
# choosing random movies
random_movie_ids = set(df_all.sample(frac=.0000001).Movie.unique())
print(random_movie_ids)

# or choosing your favorite
favorites_movie_ids = set({1, 651, 694, 1173, 1883, 4100, 5302, 8842, 13586, 13891, 15648 })
print(favorites_movie_ids)


{11810, 14725, 16390, 3626, 17482, 17295, 10162, 15829, 4565, 12501}
{15648, 1, 13891, 4100, 8842, 651, 13586, 1173, 694, 5302, 1883}


In [10]:
df_all[df_all.Movie == 17339 ]

,Movie,User,Rating
98172243,17339,1734805,3
98172244,17339,364518,4
98172245,17339,1392773,3
98172246,17339,716091,4
98172247,17339,2059652,4
...,...,...,...
98243197,17339,702455,3
98243198,17339,1536433,3
98243199,17339,1704768,3
98243200,17339,2486011,4


In [29]:
# saving the merged file to drive for further use.
movie_ids = random_movie_ids.union(favorites_movie_ids)

df_all.to_csv(path_or_buf=os.path.join(dir_name, 'data.txt.csv'),index=False)

movie_ids

{1,
 651,
 694,
 1173,
 1883,
 3626,
 4100,
 4565,
 5302,
 8842,
 10162,
 11810,
 12501,
 13586,
 13891,
 14725,
 15648,
 15829,
 16390,
 17295,
 17482}

In [30]:
# loading the saved csv into dataframe
df_all = pd.read_csv(os.path.join(dir_name, 'data.txt.csv'))

In [31]:
print("number of ratings:", len(df_all))

number of ratings: 100480507


In [14]:
df_movie_ids = df_all[df_all.Movie.isin(movie_ids)]
print("Unique movie ids in the dataframe:", (df_movie_ids.Movie.unique()))
print("Number of unique user ids:", len(df_movie_ids.User.unique()))

Unique movie ids in the dataframe: [    1   651   694  1173  1883  3626  4100  4565  5302  8842 10162 11810
 12501 13586 13891 14725 15648 15829 16390 17295 17482]
Number of unique user ids: 171266


In [15]:
df_ = df_movie_ids.sample(frac=0.001)
len(df_)

387

In [16]:
# create a dataframe that only contains rating for a subset of users and
# a subset of movies
user_ids = set(df_.User.unique())
df_small = df_all[(df_all.User.isin(user_ids) & df_all.Movie.isin(movie_ids))]
print("Unique movie ids in the dataframe:", (df_small.Movie.unique()))
print("Number of unique user ids:", len(df_small.User.unique()))
print("Number of ratings:", len(df_small))

Unique movie ids in the dataframe: [    1   651   694  1173  1883  3626  4100  4565  5302  8842 10162 11810
 12501 13586 13891 14725 15648 15829 16390 17295 17482]
Number of unique user ids: 386
Number of ratings: 1321


In [17]:
itom = list(sorted(movie_ids))
mtoi = {m: i for i, m in enumerate(itom)}
itou = list(sorted(user_ids))
utoi = {u: i for i, u in enumerate(itou)}

In [18]:
user_movie_rating = np.zeros((len(itou), len(itom)))

for index, row in df_small.iterrows():
  movie_id = row.Movie
  movie_index = mtoi[movie_id]

  user_id = row.User
  user_index = utoi[user_id]

  user_movie_rating[user_index, movie_index] = row.Rating

In [19]:
user_movie_rating.shape

(386, 21)

In [20]:
# more efficient way of building a user-movie rating matrix
um = df_small.pivot_table(index='User', columns='Movie', values='Rating').values
um.shape

(386, 21)

In [21]:
user_movie_rating[:10,]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 5.,
        0., 0., 5., 4., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5.,
        0., 1., 5., 0., 5.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3.,
        0., 0., 4., 4., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 4., 0., 0., 0., 0., 0., 5., 0., 0., 0., 3.,
        0., 0., 0., 3., 5.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 3.,
        0., 0., 0., 0., 4.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 0., 3., 0., 0., 5.,
        0., 3., 4., 4., 4.],
       [0., 0., 0., 0., 0., 3., 0., 4., 0., 0., 0., 0., 0., 0., 0., 2.,
        0., 0., 3., 0., 0.],
       [0., 0., 0., 4., 0., 0., 5., 5., 0., 0., 5., 0., 5., 0., 0., 5.,
        0., 2., 3., 4., 2.],
       [0., 0., 0., 0., 0., 4., 0., 0., 0., 0., 3., 0., 0., 0., 0., 3.,
        0., 3., 2.,

In [22]:
# compute the SVD of the matrix
A = user_movie_rating
U, S, Vt = np.linalg.svd(A, full_matrices=False)

In [23]:
print(U.shape, S.shape, Vt.shape)

(386, 21) (21,) (21, 21)


In [24]:
A_hat = U @ np.diag(S) @ Vt

In [25]:
np.set_printoptions(suppress=True, formatter={'float': '{:0.2f}'.format})
print(A_hat[:10,])

[[0.00 -0.00 0.00 -0.00 0.00 0.00 0.00 0.00 0.00 0.00 3.00 -0.00 -0.00
  -0.00 0.00 5.00 0.00 0.00 5.00 4.00 -0.00]
 [0.00 -0.00 0.00 0.00 0.00 0.00 -0.00 0.00 0.00 0.00 0.00 0.00 -0.00
  0.00 0.00 5.00 0.00 1.00 5.00 0.00 5.00]
 [0.00 -0.00 0.00 -0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 -0.00 -0.00
  0.00 -0.00 3.00 0.00 0.00 4.00 4.00 -0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 -0.00 -0.00 0.00 0.00 0.00 0.00 -0.00
  0.00 0.00 5.00 0.00 -0.00 0.00 0.00 -0.00]
 [0.00 -0.00 0.00 0.00 -0.00 4.00 0.00 0.00 0.00 -0.00 0.00 5.00 0.00
  0.00 -0.00 3.00 0.00 -0.00 0.00 3.00 5.00]
 [0.00 -0.00 0.00 0.00 0.00 0.00 -0.00 -0.00 0.00 0.00 0.00 0.00 3.00
  0.00 0.00 3.00 0.00 0.00 0.00 -0.00 4.00]
 [0.00 -0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 4.00 -0.00 3.00 0.00
  0.00 5.00 -0.00 3.00 4.00 4.00 4.00]
 [0.00 0.00 0.00 0.00 -0.00 3.00 -0.00 4.00 0.00 -0.00 0.00 0.00 -0.00
  0.00 -0.00 2.00 0.00 -0.00 3.00 0.00 -0.00]
 [-0.00 0.00 -0.00 4.00 -0.00 0.00 5.00 5.00 -0.00 -0.00 5.00 -0.00 5.00
  -0.00 

In [27]:
sum([a*a for a in S[:8]])/sum([a*a for a in S])

0.9094409763897678

In [28]:
r = 8
A_hat_r = U[:,:r] @ np.diag(S[:r]) @ Vt[:r,:]
np.set_printoptions(suppress=True, formatter={'float': '{:0.2f}'.format})
print(A_hat_r[:10,])

[[-0.14 0.01 -0.15 -0.19 0.01 0.13 0.11 0.60 -0.11 -0.02 2.89 -0.17 -0.01
  -0.04 0.02 4.96 -0.03 0.75 4.91 3.98 -0.04]
 [-0.25 0.00 -0.26 -0.43 0.01 0.01 0.49 0.48 -0.20 -0.02 -0.01 -0.13 0.00
  -0.22 0.02 4.98 -0.05 0.43 4.88 0.16 5.01]
 [-0.14 -0.00 -0.16 -0.16 -0.01 0.10 0.23 -0.07 -0.13 -0.03 0.04 -0.13
  -0.02 -0.16 0.01 2.99 -0.06 0.63 3.93 3.93 -0.03]
 [-0.02 -0.00 -0.05 -0.09 -0.01 0.00 0.02 0.23 -0.00 0.01 -0.04 0.12
  -0.00 0.02 0.00 4.98 -0.03 0.09 -0.01 0.01 -0.01]
 [0.22 0.00 0.04 0.30 0.00 4.59 0.06 -0.19 0.06 -0.01 -0.02 0.86 0.09
  0.08 0.01 3.11 0.02 0.20 -0.24 2.99 5.06]
 [-0.11 -0.00 -0.08 -0.04 -0.01 0.01 0.07 0.15 -0.10 0.01 -0.02 0.20 2.98
  -0.08 0.00 2.98 -0.04 0.28 -0.02 -0.02 3.98]
 [-0.14 0.01 -0.10 -0.01 0.02 -0.17 0.17 0.77 -0.12 -0.01 3.92 -0.02 3.04
  -0.00 0.02 5.00 -0.01 1.09 3.98 4.33 4.07]
 [0.03 0.00 -0.01 -0.02 0.01 2.86 0.44 0.37 0.01 -0.00 0.74 0.26 -0.05
  0.00 0.00 2.17 -0.00 -0.20 3.13 -0.32 0.05]
 [1.20 0.05 1.17 2.59 0.10 -0.24 4.69 1.63 1.3

In [56]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 4 Compose a query vector (here you can incorporate your taste by providing a rating for some movies and zero for unknown movies)
# ids: [1 651 694 1173 1883  3626  4100  4565  5302  8842 10162 11810 12501 13586 13891 14725 15648 15829 16390 17295 17482]
# for example: I would like rate ids 651 as 5, ids 1173 as 4, ids 4100 as 3, ids 5302 as 2, ids 15829 as 2, ids 17295 as 3, ids 17482 as 5
query_vector = np.array([0, 5, 0, 4, 0, 0, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 3, 5])

# Use compressed singular decomposition.
r = 8
Ur = U[:, :r]
Sr = np.diag(S[:r])
Vtr = Vt[:r, :]
Ur.shape
Sr.shape
Vtr.shape

print("Ur:", Ur.shape)
print("Sigmar:", Sr.shape)
print("VTr:", Vtr.shape)

# Project your query vector into concept space
query_projection = np.dot(query_vector, Vtr.T)
print("query_projection:\n", query_projection)

# Project all the users into concept space
user_concept_space = np.dot(Ur, Sr)
print("user_concept_space:\n", user_concept_space)

# Project all the movies into concept space
movie_concept_space = np.dot(Sr, Vtr)
print("movie_concept_space:\n", movie_concept_space)

# compute the similarity of query vector with all users.
user_similarities = cosine_similarity(query_projection.reshape(1, -1), user_concept_space).flatten()

# find top 5 depend on user_similarities
top_5_users_indices = np.argsort(user_similarities)[-5:][::-1]

print("indices of top 5 users:", top_5_users_indices)
print("similarities of top 5:", user_similarities[top_5_users_indices])

# compute the similarity of query vector with all movies.
movie_similarities = cosine_similarity(query_projection.reshape(1, -1), movie_concept_space.T).flatten()

# find top 5 depend on movie_similarities
top_5_movies_indices = np.argsort(movie_similarities)[-5:][::-1]

print("indices of top 5 movies:", top_5_movies_indices)
print("similarities of top 5:", movie_similarities[top_5_movies_indices])

Ur: (386, 8)
Sigmar: (8, 8)
VTr: (8, 21)
query_projection:
 [-3.93 2.93 -0.05 -1.90 1.08 0.36 1.38 5.19]
user_concept_space:
 [[-6.81 -4.33 1.42 ... 1.40 1.77 -0.63]
 [-6.86 -0.38 0.71 ... 3.30 -2.27 -0.93]
 [-4.75 -2.56 1.61 ... 1.42 2.18 -0.55]
 ...
 [-1.56 -2.39 -1.29 ... 1.30 0.06 -0.67]
 [-3.69 -3.43 -0.28 ... -1.30 1.51 -0.53]
 [-3.18 1.21 0.30 ... -1.60 -1.82 0.02]]
movie_concept_space:
 [[-1.39 -0.08 -1.03 -4.62 -0.16 -18.83 -11.55 -4.66 -1.25 -0.17 -17.18
  -3.82 -32.61 -1.62 -0.14 -51.99 -0.26 -7.74 -36.33 -35.09 -37.84]
 [-0.14 -0.07 -0.11 0.67 -0.13 -6.12 -1.60 -3.65 -0.37 0.01 -13.63 1.34
  14.72 -0.47 -0.04 -2.67 -0.21 1.08 -23.61 0.39 23.06]
 [-0.11 -0.07 -1.04 -1.79 -0.14 4.38 -5.10 -3.22 -0.59 -0.20 -11.19 1.20
  -21.46 -0.68 0.00 19.22 -0.29 0.06 -11.61 11.63 -2.53]
 [1.71 -0.04 1.81 4.14 -0.08 -1.48 6.61 -1.10 1.93 0.30 -6.91 1.20 17.68
  1.70 -0.09 9.20 0.15 0.24 -7.39 2.62 -22.13]
 [0.34 -0.07 -0.03 0.81 -0.14 10.54 5.59 -4.70 0.15 -0.08 -23.52 0.91
  0.11 -0.89 -0